## This Notebook will create a GMM to classify shots into one of the 6 actions

In [1]:
import PlottingFunction as pf
import DataManipulation as dm
import matplotlib.pyplot as plt
import Policy_iteration_functions as f
from pathlib import Path
import pandas as pd
import numpy as np
from scipy.stats import multivariate_normal
import winsound

%matplotlib qt5

In [2]:
df  = pd.read_pickle(Path.cwd() / 'pickle' / 'TP' / 'df_double_prime.plk')
df = df[df.second_error != 'none']
df = df.reset_index(drop=True)
df = dm.add_landing_region_to_df(df, more_actions = True)

In [3]:
proportions, weights_rally, weights_deuce_serve, weights_ad_serve = dm.calculate_weights(df)

In [5]:
region_labels = proportions.index.values
df_region_dict = {}
distribution_region_dict = {}

eps = 0.00000000001

for region_label in region_labels:
    # slice for particular region/action
    df_region = df[df.region == region_label]
    # check if theres more than 20,000, if so, sample just 20,000
    if df_region.shape[0] > 20000:
        df_region = df_region.sample(20000, random_state=0)
    
    # save the subset of shots
    df_region_dict[region_label] = df_region
    
    # fit and save the gaussian distribution
    X = df_region[['x02', 'y02']]
    X = X.to_numpy()
    
    mean = np.mean(X, axis=0)
    cov = np.cov(X, rowvar=0)
    # Add epsilon error
    cov *= eps
    
    distribution = multivariate_normal(mean=mean, cov=cov)
    distribution_region_dict[region_label] = distribution

# del df, df_region, X

In [6]:
if eps != 1:
    df_eps = pd.DataFrame(columns=['x02','y02'])

    for region_label in region_labels[0:25]:
        n = proportions[region_label]
        sim_shots = (distribution_region_dict[region_label].rvs(size=n))
        df_eps = df_eps.append(pd.DataFrame(sim_shots, columns=['x02','y02']))

    df_eps = dm.determine_weights_for_epsilon_distributions(df_eps, more_actions = True)
    proportions_eps, weights_rally, _ , _ = dm.calculate_weights(df_eps)

In [14]:
# df = dm.actionProbabilityVector(df,distribution_region_dict, region_labels, weights_rally, weights_deuce_serve, weights_ad_serve)
# count = 0
# probableAction = []

# for index, shot in df.iterrows(): # loop through the rows for the particular state
#     if(shot['OBDrop':].sum() == 0):
        # probableAction.append(np.random.choice(['OBDrop','adOBBase', 'adOBDeepSide','adOBShortSide','deuceOBBase', 'deuceOBDeepSide','deuceOBShortSide'], p =[1/7,1/7,1/7,1/7,1/7,1/7,1/7]))
        # count = count + 1
    # else:
    #     probableAction.append(np.random.choice(region_labels, p = shot['OBDrop':]))

# df['probableAction'] = probableAction

# # Save the df with probable actions
# df.to_pickle('df_eps0_action.plk')
# # Read the probable actions df
# df  = pd.read_pickle('df_eps_action.plk')

In [5]:
df  = pd.read_pickle('df_eps0.01_action.plk')

yo = df.iloc[:, 83:112].copy()
del df

TypeError: cannot do slice indexing on <class 'pandas.core.indexes.base.Index'> with these indexers [OBDrop] of <class 'str'>

In [9]:
# df  = pd.read_pickle('df_eps0.5_action.plk')
df.Error.sum()
# df = df.sample(100)

# proportions_sam = df.groupby('probableAction').count().iloc[:, 0]
# total_rally_sam = proportions_sam['OBDrop':'deuceShortSide'].sum()
# weights_rally_sam = proportions_sam['OBDrop':'deuceShortSide'] / total_rally_sam
# # 
# # df = df.sample(100)
# df.probableAction.str.count("OB").sum()
# # 
# totOB = 0
# for name, value in enumerate(weights_rally_sam):
#     if 'OB' in region_labels[name]:
#         totOB += value       
# totOB

119601

In [10]:

for index, shot in df.iterrows(): # loop through the rows for the particular state
    if 'OB' in shot['probableAction']:
        df.at[index, 'Error'] = True
        
df.Error.sum()

136608

In [9]:
del df


In [ ]:
# df_compare = df.groupby(["region", "probableAction"]).size().reset_index(name="Time")
# df_compare = df_compare[df_compare.region != df_compare.probableAction]
# percent_diff = df_compare['Time'].sum()/871869*100
# winsound.Beep(2000,800)
# # del df
# percent_diff

In [11]:
shot = 192549
df_sample = df.loc[shot,:]
plt.figure()
pf.plot1D(df_sample, compressed=True, only_half=False, big_buckets=True)
pf.createCourt(plt.gca(), compressed=True, only_half=False, big_buckets=True)

In [ ]:
x, y = np.mgrid[0:16:0.1, -7:7:0.1]
pos = np.dstack((x, y))
z = 0

for label, distribution in distribution_region_dict.items():
    if ('deuceBase' in label) or ("deuceOBBase" in label):
        z += distribution.pdf(pos)
        
fig = plt.figure()
plt.subplot(1,1,1)
plt.contourf(x, y, z,zorder=2)
pf.createCourt(plt.gca(), compressed=True, only_half=False, big_buckets=True)
pf.plot1D(df_sample, compressed=True, only_half=False, big_buckets=True)


In [ ]:
shot = 245642
# shot = 255937
# shot = 272631


x, y = np.mgrid[0:16:0.1, -7:7:0.1]
pos = np.dstack((x, y))
zDeep = 0

for label, distribution in distribution_region_dict.items():
    if 'serve' not in label:
        if ('Deep' in label) or ("Base" in label):
            zDeep += distribution.pdf(pos)
    
fig = plt.figure()
plt.subplot(111)
plt.contourf(x, y, zDeep,zorder=2)
pf.createCourt(plt.gca(), compressed=True, only_half=False, big_buckets=True)

df_sample = df.loc[shot,:]
pf.plot1D(df_sample,compressed=True, only_half=False,big_buckets=True,)

# del df

### Plotting all the KDE's and Distributions

In [ ]:
x, y = np.mgrid[0:16:0.1, -7:7:0.1]
pos = np.dstack((x, y))
zShort = 0; zDeep = 0

for label, distribution in distribution_region_dict.items():
    if 'serve' not in label:
        if "OB" not in label:
            if ('Deep' in label) or ("Base" in label):
                zDeep += distribution.pdf(pos)
            elif 'Corner' not in label:
                zShort += distribution.pdf(pos)

zDeep += zShort

fig = plt.figure()
plt.subplot(2,1,1)
plt.contourf(x, y, zDeep,zorder=2)
pf.createCourt(plt.gca(), compressed=True, only_half=False, big_buckets=True)
plt.title('eps = 1')
plt.subplot(2, 1, 2)
plt.contourf(x, y, zDeep,zorder=2)
pf.createCourt(plt.gca(), compressed=True, only_half=False, big_buckets=True)
plt.title('eps = 3')
# plt.subplot(3, 1, 3)
# plt.contourf(x, y, zDeep,zorder=2)
# pf.createCourt(plt.gca(), compressed=True, only_half=False, big_buckets=True)
# plt.title('eps = 5')
# plt.subplot(2, 2, 4)
# plt.contourf(x, y, zDeep,zorder=2)
# pf.createCourt(plt.gca(), compressed=True, only_half=False, big_buckets=True)
# plt.title('eps = 7')

In [ ]:
# cmaps = ["Blues", "Reds", "Oranges", "Greens", "Purples", "Greys", "Blues", "Reds", "Oranges", "Greens", "Purples", "Greys"]
# 
# plt.figure()
# i = 0
# for label, df_region in df_region_dict.items():    
#     if 'serve' not in label:
#         if ('Deep' in label) or ("Base" in label):
#             X = df_region[['x02', 'y02']]
#             X = X.to_numpy()
#             # pf.createKde(X[:,0],X[:,1],compressed=True, only_half=False,big_buckets=True, n_levels=15, cmap=cmaps[i])
#             # i = i+1
#         elif 'Corner' not in label:
#             if "OBDrop" not in label:
#                 X = df_region[['x02', 'y02']]
#                 X = X.to_numpy()
#                 pf.createKde(X[:,0],X[:,1],compressed=True, only_half=False,big_buckets=True, n_levels=15, cmap=cmaps[i])
#                 i = i+1


